Credit where credit is due: this is based on  https://www.kaggle.com/code/gpreda/fast-test-of-llama-v2-pre-quantized-with-llama-cpp.

Loading the model will take a while so please be patient

In [1]:
# clean up after failed efforts if necessary
!rm -rf llama.cpp
# we need no CUDA for CPU; this flag has changed since the initial release
!CMAKE_ARGS="-DLLAMA_CUDA=off" pip install llama-cpp-python 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 21.0 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / - \ | done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.78-cp310-cp310-linux_x86_64.whl size=3658465 sha256=501a80a6c19ac98914810414df87060c335f6593c8dd3b649273a7f056cb4a58
  Stored in directory: /root/.cache/pip/wheels/fd/c5/bd/3b1c20081bd71ce9d28b562573c97915c790bf1ef231879a61
Successfully built llama-cpp-python


In [2]:
import arrow
from llama_cpp import Llama

time_load = arrow.now()
# llm = Llama(model_path="/kaggle/input/test-llama-2-quantized-with-llama-cpp/llama-7b.gguf")
llm = Llama(
    model_path="/kaggle/input/meta-llama-3-8b-gguf/llama3-8b-instruct-Q5_K_M.gguf",
    n_ctx=4096,      # Max tokens for in + out
    n_threads=4,     # CPU cores used
    n_gpu_layers=-1,  # Load all layers into VRAM of the GPU
)
print("Model load time: {:5.4f} seconds".format((arrow.now() - time_load).total_seconds()))

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /kaggle/input/meta-llama-3-8b-gguf/llama3-8b-instruct-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   7:                 lla

Model load time: 65.9038 seconds


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 
Model metadata: {'tokenizer.chat_template': "{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}", 'tokenizer.ggml.eos_token_id': '128009', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'gpt2', 'general.architecture': 'llama', 'llama.rope.freq_base': '500000.000000', 'llama.context_length': '8192', 'general.name': '.', 'llama.vocab_size': '128256', 'general.file_type': '17', 'llama.embedding_length': '4096', 'lla

In [3]:
def ask(query: str, max_tokens: int):
    time_ask = arrow.now()
    output = llm("Q: {} A: ".format(query), max_tokens=max_tokens, stop=["Q:", "\n"], echo=False)
    return [output['choices'][index]['text'] for index in range(len(output['choices']))], (arrow.now() - time_ask).total_seconds()

query_cities = 'What are the names of three European capital cities?'
result_cities, time_cities = ask(query=query_cities, max_tokens=40)
print(query_cities)
print(result_cities)
print('time: {:5.4f}'.format(time_cities))


llama_print_timings:        load time =    4843.59 ms
llama_print_timings:      sample time =      51.08 ms /    22 runs   (    2.32 ms per token,   430.71 tokens per second)
llama_print_timings: prompt eval time =    4843.47 ms /    16 tokens (  302.72 ms per token,     3.30 tokens per second)
llama_print_timings:        eval time =    7049.44 ms /    21 runs   (  335.69 ms per token,     2.98 tokens per second)
llama_print_timings:       total time =   11978.64 ms /    37 tokens


What are the names of three European capital cities?
['1) Paris, France; 2) London, United Kingdom; and 3) Rome, Italy.']
time: 11.9889


In [4]:
query_catcher = 'What can you tell me about the J. D. Salinger novel The Catcher in the Rye?'
result_catcher, time_catcher = ask(query=query_catcher, max_tokens=1000)
print(query_catcher)
print(result_catcher)
print('time: {:5.4f}'.format(time_catcher))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4843.59 ms
llama_print_timings:      sample time =     576.18 ms /   234 runs   (    2.46 ms per token,   406.13 tokens per second)
llama_print_timings: prompt eval time =    6029.25 ms /    24 tokens (  251.22 ms per token,     3.98 tokens per second)
llama_print_timings:        eval time =   82581.36 ms /   233 runs   (  354.43 ms per token,     2.82 tokens per second)
llama_print_timings:       total time =   89579.45 ms /   257 tokens


What can you tell me about the J. D. Salinger novel The Catcher in the Rye?
['1) Published in 1951, this coming-of-age story follows the protagonist, Holden Caulfield, as he navigates his way through a challenging day in New York City after being expelled from a prestigious boarding school. Throughout the novel, Holden grapples with feelings of alienation, rebellion, and disillusionment, leading him to question the phoniness of the adult world and the superficiality of those around him. 2) The title "The Catcher in the Rye" is derived from a line in Robert Burns\' poem "Comin\' Through the Rye," which Holden misquotes as "If a body catch a body coming through the rye." This phrase symbolizes his desire to protect children from falling off a cliff, representing his fear of losing innocence and becoming corrupt like the adults he despises. 3) The novel has been widely interpreted as an exploration of adolescent angst, exploring themes such as identity formation, rebellion against societa

In [5]:
r, t = ask(query='How do I drive from Washington DC to New York City?', max_tokens=3000)
print(r)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4843.59 ms
llama_print_timings:      sample time =     106.38 ms /    48 runs   (    2.22 ms per token,   451.20 tokens per second)
llama_print_timings: prompt eval time =    3792.90 ms /    15 tokens (  252.86 ms per token,     3.95 tokens per second)
llama_print_timings:        eval time =   15883.38 ms /    47 runs   (  337.94 ms per token,     2.96 tokens per second)
llama_print_timings:       total time =   19846.21 ms /    62 tokens


["\xa0Driving from Washington, D.C. to New York City can be a bit of a challenge due to traffic and construction, but it's definitely doable. Here are the steps you can follow to make your journey as smooth as possible:"]


In [6]:
r0, t0 = ask(query='What would I need to do to build an edible bicycle?', max_tokens=3000)
print(r0)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4843.59 ms
llama_print_timings:      sample time =      57.51 ms /    26 runs   (    2.21 ms per token,   452.13 tokens per second)
llama_print_timings: prompt eval time =    3792.21 ms /    15 tokens (  252.81 ms per token,     3.96 tokens per second)
llama_print_timings:        eval time =    9657.58 ms /    25 runs   (  386.30 ms per token,     2.59 tokens per second)
llama_print_timings:       total time =   13539.82 ms /    40 tokens


["1. Choose the right materials: You'll want a durable and food-safe material for your bicycle, like stainless steel or bamboo."]
